#  Online Learning Rate in Keras
> compatible with model.fit
- toc: true
- branch: master
- badges: true
- image: images/dynamic-pad.png
- comments: true
- author: Sajjad Ayoubi
- categories: [tips]

- I use this functionality with `Tensorboard`
  - we change the LR with regard to the loss curve in Tensorboard

In [1]:
import pandas as pd
import tensorflow as tf

- we create a CSV file for reading lrs
  - we need a Keras scheduler in order to use model.fit

In [20]:
class OnlineLr():
  def __init__(self, init_lrs=[5e-3], path='lrs'):
      # create a CSV file for saving lrs
      lrs = pd.DataFrame(data=init_lrs, columns=['lrs'])
      lrs.index.name = 'epoch'
      self.path = path+'.csv'
      lrs.to_csv(self.path)
      del lrs
  
  def add_lrs(self, lr):
      df = pd.read_csv(self.path)
      df = df.append({'epoch': df.epoch.iloc[-1], 'lrs': lr}, ignore_index=True)
      df.to_csv(self.path)
      del df

  def __call__(self, epoch):
      # give an epoch, return a lr
      df = pd.read_csv(self.path)
      return df.lrs[epoch]

In [24]:
scheduler = OnlineLr(init_lrs=[5e-3])
scheduler(0) # 5e-5

0.005

- we can add new lr in another kernel 
  - or dirctly from file

In [25]:
scheduler.add_lrs(lr=1e-3)
scheduler(1)

0.001

- with `keras.callbacks` we can use it in `model.fit`

In [23]:
callback = tf.keras.callbacks.LearningRateScheduler(schedule=scheduler, verbose=1)
# then pass it to the fit function

- alternative choice is `LROnPlateau`
  - but I love to have the most control